In [ ]:
import sys

import numpy as np
import tensorflow as tf

In [ ]:
print('TF version: {}'.format(tf.version.VERSION))
print('PY version: {}'.format(sys.version))
print('Eager mode: {}'.format(tf.executing_eagerly()))

In [ ]:
data = tf.data.Dataset.from_tensor_slices(list(range(100)))

## Iterators
- `make_one_shot_iterator`: Initialization is not necessary.
- `make_initializable_iterator`: Initializable.

In [ ]:
it = data.make_one_shot_iterator()
nx = it.get_next()  # Tensor

sess = tf.Session()
out = []
while True:
  try:
    out.append(sess.run(nx))
  except tf.errors.OutOfRangeError:
    break
print(np.array(out).reshape(10, -1))

In [ ]:
it = data.make_initializable_iterator()  # Operator
nx = it.get_next()  # Tensor

sess = tf.Session()
for i in range(2):
  sess.run(it.initializer)
  out = []
  for i in range(10):
    out.append(sess.run(nx))
  print(out)

## Shuffle

In [ ]:
shuffled = data.shuffle(10)
it = shuffled.make_initializable_iterator()
nx = it.get_next()

with tf.Session() as sess:
  sess.run(it.initializer)
  out = []
  for i in range(100):
    out.append(sess.run(nx))
  print(np.array(out).reshape((10, -1)))

## Batch

In [ ]:
batch = data.batch(30, drop_remainder=False)
it = batch.make_initializable_iterator()
nx = it.get_next()

sess = tf.Session()
sess.run(it.initializer)
for i in range(4):
  print(sess.run(nx))

### with `drop_remainder = True`

In [ ]:
batch = data.batch(30, drop_remainder=True)
it = batch.make_initializable_iterator()
nx = it.get_next()

sess = tf.Session()
sess.run(it.initializer)
try:
  for i in range(4):
    print('{} ==> {}'.format(i, sess.run(nx)))
except tf.errors.OutOfRangeError as e:
  print('Out of range at {}'.format(i))

### repeat & batch

In [ ]:
batch = data.batch(30, drop_remainder=True).repeat(2)
it = batch.make_initializable_iterator()
nx = it.get_next()

sess = tf.Session()
sess.run(it.initializer)
try:
  i = 0
  while True:
    print('{} ==> {}'.format(i, sess.run(nx)))
    i += 1
except tf.errors.OutOfRangeError as e:
  print('Out of range at {}'.format(i))

In [ ]:
# Repeat then batch
batch = data.repeat(2).batch(30, drop_remainder=True)
it = batch.make_initializable_iterator()
nx = it.get_next()

sess = tf.Session()
sess.run(it.initializer)
try:
  i = 0
  while True:
    print('{} ==> {}'.format(i, sess.run(nx)))
    i += 1
except tf.errors.OutOfRangeError as e:
  print('Out of range at {}'.format(i))

## From generator

In [ ]:
def mygen(n):
  for i in range(n):
    print('yield {}'.format(i))
    yield i

# The first argument `generator` is a factory of a generator.
data = tf.data.Dataset.from_generator(mygen, output_types=tf.int32, args=(5,)).repeat(2)
sess = tf.Session()

it = data.make_one_shot_iterator()
nx = it.get_next()
try:
  while True:
    print('value = {}'.format(sess.run(nx)))
except tf.errors.OutOfRangeError:
  pass